In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import nltk
import re
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.combine import SMOTETomek
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

In [2]:
#read data
data=pd.read_excel('1000 leads.xlsx')

In [3]:
len(data['Location'].unique())

56

In [4]:
data['Status '].replace(to_replace="NOt Converted",value="Not Converted",inplace=True)
data['Status '].replace(to_replace='Converted ',value="Converted",inplace=True)
data['Status '].replace(to_replace='Conveted',value="Converted",inplace=True)

In [5]:
data.drop(columns=['Unnamed: 4','Lead Name'],inplace=True)
data.dropna(inplace=True)

In [6]:
data.isna().sum()

Location              0
Status                0
Status information    0
dtype: int64

In [7]:
data.reset_index(inplace=True)

In [8]:
df=data

In [9]:
import re
import nltk

from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download(['wordnet','stopwords'] )

  
#ps = PorterStemmer()
wordnet=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to C:\Users\mohammed
[nltk_data]     adil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\mohammed
[nltk_data]     adil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
corpus = []

for i in range(0,len(data)):
    review=re.sub('[^a-zA-Z]',' ',data['Status information'][i])
    review=review.lower()
    review=review.split()

    review = [wordnet.lemmatize(word) for word in review if not word in stopwords.words('english')] 
    # removing stop words and also lematizing
    review=" ".join(review)
    corpus.append(review)

In [11]:
for i in range(0 ,len(corpus)):
    if "prema" in corpus[i]:
        corpus[i]=corpus[i].replace("prema",'',True)
    elif "mohan" in corpus[i]:
        corpus[i]=corpus[i].replace("mohan",'',True)
    elif "gowtham" in corpus[i]:
        corpus[i]=corpus[i].replace("gowtham",'',True)
    elif "surendra" in corpus[i]:
        corpus[i]=corpus[i].replace("surendra",'',True)
    elif "soma" in corpus[i]:
        corpus[i]=corpus[i].replace("soma",'',True)
    else:
        corpus[i]=corpus[i]

In [12]:
#cv = CountVectorizer(max_features=2500)
#X = cv.fit_transform(corpus).toarray()

tv = TfidfVectorizer(max_features=3000,ngram_range=(3,6))      
X = tv.fit_transform(corpus).toarray()      

In [13]:
y=pd.get_dummies(data['Status ']) #onehot coding for label column into y variable
# Not converted : 1
#converted :0

In [14]:
y=y.iloc[:,1].values 
y

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [15]:
smk = SMOTETomek(random_state=1234)
x_new, y_new = smk.fit_resample(X,y)
x_new.shape, y_new.shape

((1654, 3000), (1654,))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(x_new, y_new, test_size = 0.20, random_state = 0)

In [17]:
model = SVC(kernel = 'rbf', random_state = 0).fit(X_train, y_train)
y_pred=model.predict(X_test)
print(accuracy_score(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))
print(classification_report(y_pred,y_test))

0.8882175226586103
[[166  33]
 [  4 128]]
              precision    recall  f1-score   support

           0       0.98      0.83      0.90       199
           1       0.80      0.97      0.87       132

    accuracy                           0.89       331
   macro avg       0.89      0.90      0.89       331
weighted avg       0.90      0.89      0.89       331



## Task :2


In [18]:
import re
import nltk

from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download(['wordnet','stopwords'] )

  
#ps = PorterStemmer()
wordnet=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to C:\Users\mohammed
[nltk_data]     adil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\mohammed
[nltk_data]     adil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
corpus1 = []

for i in range(0,len(df)):
    review1=re.sub('[^a-zA-Z]',' ',df['Status information'][i])
    review1=review1.lower()
    review1=review1.split()

    review1 = [wordnet.lemmatize(word) for word in review1 if not word in stopwords.words('english')] 
    # removing stop words and also lematizing
    review1=" ".join(review1)
    corpus1.append(review1)

In [20]:
corpus2 = []

for i in range(0,len(df)):
    review2=re.sub('[^a-zA-Z]',' ',df['Location'][i])
    review2=review2.lower()
    review2=review2.split()

    review2 = [wordnet.lemmatize(word) for word in review2 if not word in stopwords.words('english')] 
    # removing stop words and also lematizing
    review2=" ".join(review2)
    corpus2.append(review2)

In [21]:
BE=[]
for i in range(0 ,len(corpus1)):
    if "prema" in corpus1[i]:
        BE.append('prema')
        corpus1[i]=corpus1[i].replace("prema",'',True)
    elif "mohan" in corpus1[i]:
        BE.append('mohan')
        corpus1[i]=corpus1[i].replace("mohan",'',True)
    elif "gowtham" in corpus1[i]:
        BE.append('gowtham')
        corpus1[i]=corpus1[i].replace("gowtham",'',True)
    elif "surendra" in corpus1[i]:
        BE.append('surendra')
        corpus1[i]=corpus1[i].replace("surendra",'',True)
    elif "soma" in corpus1[i]:
        BE.append('soma')
        corpus1[i]=corpus1[i].replace("soma",'',True)
    else:
        BE.append('others')
        corpus1[i]=corpus1[i]

In [22]:
len(BE)
df['BE']=BE

In [23]:
corpus3 = []

for i in range(0,len(df)):
    review3=re.sub('[^a-zA-Z]',' ',df['BE'][i])
    review3=review3.lower()
    review3=review3.split()

    review3 = [wordnet.lemmatize(word) for word in review3 if not word in stopwords.words('english')] 
    # removing stop words and also lematizing
    review3=" ".join(review3)
    corpus3.append(review3)

In [24]:
#cv = CountVectorizer(max_features=2500)
#X = cv.fit_transform(corpus).toarray()

tv1 = TfidfVectorizer(max_features=3000,ngram_range=(3,6))      #defining tfidf vec for transformation
x1 = tv1.fit_transform(corpus1).toarray()#transforming our preprocesssed data to vectors adn array
tv2 = TfidfVectorizer(max_features=3000,ngram_range=(1,2))      #defining tfidf vec for transformation
x2 = tv2.fit_transform(corpus2).toarray()
tv3 = TfidfVectorizer(max_features=3000,ngram_range=(1,2))
x3 = tv3.fit_transform(corpus3).toarray()

In [25]:
y=pd.get_dummies(df['Status ']) #onehot coding for label column into y variable
# Not converted : 1
#converted :0
y=y.iloc[:,1].values 
y

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [26]:
x=np.concatenate((x1,x2,x3),axis=1)

In [27]:
smk = SMOTETomek(random_state=1234)
x, y = smk.fit_resample(x,y)
x.shape, y.shape

((1658, 3051), (1658,))

In [28]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [29]:
model = SVC(kernel = 'rbf', random_state = 0).fit(X_train, y_train)
y_pred=model.predict(X_test)
print(accuracy_score(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))
print(classification_report(y_pred,y_test))

0.9066265060240963
[[156  24]
 [  7 145]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91       180
           1       0.86      0.95      0.90       152

    accuracy                           0.91       332
   macro avg       0.91      0.91      0.91       332
weighted avg       0.91      0.91      0.91       332

